In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np


## Tensorflow Basic : Operations

In [2]:
# load data
dataset = tfds.load('cifar100')
trainset, testset = dataset['train'], dataset['test']

# create variables
var1 = tf.Variable([1.0, 2.0])
var2 = tf.Variable(tf.random.truncated_normal(shape=[2], mean=0.0, stddev=1.0,))

# convolution operation
fltr_height, fltr_weight, in_channels, out_channels = 2, 2, 3, 5
fltr = tf.Variable(tf.random.truncated_normal(shape=[fltr_height, fltr_weight, in_channels, out_channels]))
batch_size, img_height, img_weight, img_channels = 32, 10, 10, 3
ipts = np.random.rand(batch_size, img_height, img_weight, img_channels)
opts = tf.nn.conv2d(ipts, filters=fltr, padding='SAME', strides=1)

# print
print('input shape : ', tf.shape(ipts).numpy(), ' filter shape : ', tf.shape(fltr).numpy(), ' output shape : ', tf.shape(opts).numpy())

input shape :  [32 10 10  3]  filter shape :  [2 2 3 5]  output shape :  [32 10 10  5]


## Tensorflow Module : Container for variables and other modules

In [3]:
class SubModule(tf.Module):
    def __init__(self, val1, val2):
        self.var1 = tf.Variable(val1)
        self.var2 = tf.Variable(val2)
        
class DummyModule(tf.Module):
    def __init__(self):
        self.var1 = tf.Variable(1)
        self.var2 = tf.Variable(2)
        self.sub1 = SubModule(3, 4)
        self.sub2 = SubModule(5, 6)
        return
    
model = DummyModule()
print('DummyModule.trainable_variables')
for var in model.trainable_variables:
    print(var.numpy())

print('DummyModule.submodules')
for submodule in model.submodules:
    print([var.numpy() for var in submodule.trainable_variables])

DummyModule.trainable_variables
1
2
3
4
5
6
DummyModule.submodules
[3, 4]
[5, 6]


## Calculate gradient

In [4]:
# datas
datanum, iptdim, optdim = 10, 3, 2
x, y = np.random.rand(datanum, iptdim), np.random.rand(datanum, optdim)

# variables 
w = tf.Variable(tf.random.normal(shape=[iptdim, optdim], dtype=tf.float64))
b = tf.Variable(tf.random.normal(shape=[optdim], dtype=tf.float64))
variables = [w, b]

# calculate derivative
with tf.GradientTape() as tape:
    pred = tf.matmul(x, w) + b
    loss = tf.reduce_mean(tf.reduce_sum((y - pred) ** 2, axis=1))
gradients = tape.gradient(loss, variables)

## Apply gradient

In [5]:
# apply gradient
original_variables = [var.numpy() for var in variables]
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)
optimizer.apply_gradients(zip(gradients, variables))

# check direction
for idx in range(len(gradients)):
    grad, var = gradients[idx], variables[idx]
    diff = var - original_variables[idx]
    dot = tf.reduce_sum(grad * diff)
    norm = tf.norm(grad) * tf.norm(diff)
    print(dot / norm)

tf.Tensor(-1.0000000000000002, shape=(), dtype=float64)
tf.Tensor(-1.0, shape=(), dtype=float64)


## Keras model : Easy way to build and train deep learning model

In [6]:
# datas
clsnum = 10
datanum = 10
ipt_shape = [10, 10, 3]
x, y = np.random.rand(datanum, *ipt_shape), np.random.randint(high=clsnum, low=0, size=datanum)

# model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(
        filters=7,
        kernel_size=[2, 2],
        strides=1,
        padding='SAME'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'), 
    tf.keras.layers.Dropout(rate=0.2), 
    tf.keras.layers.Conv2D(
        filters=clsnum,
        kernel_size=[2, 2],
        strides=1,
        padding='SAME'),
    tf.keras.layers.GlobalMaxPool2D(), 
    tf.keras.layers.Softmax()
])

# compile and 
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss_fn)

# train (fit)
history = model.fit(x=x, y=y, epochs=5, verbose=0) # verbose=1 : print training state
print(history.history["loss"])

[2.6351914405822754, 2.5531158447265625, 2.55641770362854, 2.6709680557250977, 2.481041669845581]


## Save and restore variables

In [7]:
class DummyModule(tf.Module):
    def __init__(self, val):
        self.var = tf.Variable(val)
        
# save
v1 = tf.Variable(1)
m1 = DummyModule(1)
ckpt_map1 = {'var' : v1, 'model' : m1}
ckpt1 = tf.train.Checkpoint(**ckpt_map1)
ckpt_manager = tf.train.CheckpointManager(checkpoint=ckpt1, directory='ckpt', max_to_keep=1)
ckpt_manager.save()

# make other variables
v2 = tf.Variable(2)
m2 = DummyModule(2)
print('before restore', v2.numpy(), m2.var.numpy())

# restore
ckpt_map2 = {'var' : v2, 'model' : m2}
ckpt2 = tf.train.Checkpoint(**ckpt_map2)
ckpt2.restore(ckpt_manager.latest_checkpoint)
print('after restore', v2.numpy(), m2.var.numpy())

before restore 2 2
after restore 1 1


## Save variables and operation graph of model 

In [8]:
MODEL_PATH = 'saved_model'
datacnt, iptdim, optdim = 32, 5, 3
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(optdim, input_shape=[None, iptdim]))

# compile
model.compile()

# save model
model.save(MODEL_PATH)

# load model
loaded = tf.keras.models.load_model(MODEL_PATH)

# compare
x = np.random.rand(datacnt, iptdim)
print(tf.reduce_sum(tf.abs(loaded(x) - model(x))).numpy())

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


0.0
